In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os

In [2]:
BATCH_SIZE = 64
EPISODE = 500000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Nom = 1e-5
Weight_Deacy =1e-5
GPU = 0
Margin = 1

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'AwA2_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_2 import AttributeNetwork
from my_net_2 import TripletNetwork_cos

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(85,1024,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
triplet_network = TripletNetwork_cos(attribute_network)  #metric learning   
triplet_network.cuda(GPU)   
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
#triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE)
#
triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE, 
                                         weight_decay=Weight_Deacy)
triplet_network_scheduler = StepLR(triplet_network_optim , step_size=100000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*1024

        #print(batch_features_ext)

        sample_features = attribute_network(Variable(sample_attributes).cuda(GPU).float())
        sample_features_ext = sample_features.repeat(batch_size, 1)
        #print(sample_features_ext.shape)


        relations = F.cosine_similarity(batch_features_ext, sample_features_ext).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        #print(predict_labels)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
print("training...")
last_accuracy = 0.0
for episode in range(EPISODE):
    #attribute_network.train()
    triplet_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    
    batch_features_negative = np.empty_like(batch_features.numpy())
    
    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_labels.numpy()]).squeeze(1)
    
    batch_features = Variable(batch_features).cuda(GPU).float()  # 32*2048
    
    for i in range(batch_features_negative.shape[0]):
        pick=np.random.choice(np.where(batch_labels.numpy()[i]!=train_id)[0], replace=True)
        batch_features_negative[i] = x[pick]    
    
    batch_features_negative = torch.from_numpy(batch_features_negative)
    batch_features_negative = Variable(batch_features_negative).cuda(GPU).float()  # 32*2048
    
    #batch_attributes = attribute_network(Variable(batch_attributes).cuda(GPU))
    batch_attributes = Variable(batch_attributes).cuda(GPU)
    
    dista, distb, embedded_x, embedded_y, embedded_z = triplet_network(batch_attributes, batch_features, batch_features_negative)
    
    criterion = nn.MarginRankingLoss(margin = Margin).cuda(GPU)
    
    target = torch.FloatTensor(dista.size()).fill_(-1).cuda(GPU)
    
    
    loss_triplet= criterion(dista, distb, target)
    loss_embedd = embedded_x.norm(2) + embedded_y.norm(2) + embedded_z.norm(2)
    loss = loss_triplet + Nom * loss_embedd

    # update
    triplet_network.zero_grad()
    loss.backward()
    triplet_network_optim.step()
    
    if (episode+1)%1000 == 0:
        print("episode:", episode+1, "loss", loss.data)
    if (episode+1)%1000 == 0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy(test_features, test_label, np.arange(50), attributes)
        gzsl_seen_accuracy = compute_accuracy(test_seen_features, test_seen_label, np.arange(50), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        print('zsl:', zsl_accuracy)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        print('_'*100)

training...
episode: 1000 loss tensor(0.9274, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.03917603942878807
gzsl: unseen=0.0000 , seen=0.0085 , h=0.0000
____________________________________________________________________________________________________
episode: 2000 loss tensor(0.9207, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.03917603942878807
gzsl: unseen=0.0000 , seen=0.0172 , h=0.0000
____________________________________________________________________________________________________
episode: 3000 loss tensor(0.9291, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.0398079110324782
gzsl: unseen=0.0000 , seen=0.0172 , h=0.0000
____________________________________________________________________________________________________
episode: 4000 loss tensor(0.9196, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.1431821053961835
gzsl: unseen=0.0000 , seen=0.0496 , 

episode: 31000 loss tensor(0.7986, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.4980411980285606
gzsl: unseen=0.1228 , seen=0.3484 , h=0.1816
____________________________________________________________________________________________________
episode: 32000 loss tensor(0.7968, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5066346518387463
gzsl: unseen=0.1285 , seen=0.3552 , h=0.1887
____________________________________________________________________________________________________
episode: 33000 loss tensor(0.8262, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5062555288765322
gzsl: unseen=0.1256 , seen=0.3591 , h=0.1861
____________________________________________________________________________________________________
episode: 34000 loss tensor(0.8161, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5057500315935802
gzsl: unseen=0.1341 , seen=0.3677 , h=0.1965
_

episode: 61000 loss tensor(0.7579, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5300139011752811
gzsl: unseen=0.1362 , seen=0.4871 , h=0.2129
____________________________________________________________________________________________________
episode: 62000 loss tensor(0.8140, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5263490458738784
gzsl: unseen=0.1395 , seen=0.4884 , h=0.2170
____________________________________________________________________________________________________
episode: 63000 loss tensor(0.7730, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5306457727789713
gzsl: unseen=0.1415 , seen=0.4896 , h=0.2196
____________________________________________________________________________________________________
episode: 64000 loss tensor(0.7980, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5295084038923291
gzsl: unseen=0.1404 , seen=0.4954 , h=0.2188
_

episode: 91000 loss tensor(0.7939, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5398710981928472
gzsl: unseen=0.1650 , seen=0.5451 , h=0.2534
____________________________________________________________________________________________________
episode: 92000 loss tensor(0.7537, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5410084670794895
gzsl: unseen=0.1683 , seen=0.5481 , h=0.2576
____________________________________________________________________________________________________
episode: 93000 loss tensor(0.7730, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5401238468343232
gzsl: unseen=0.1685 , seen=0.5468 , h=0.2576
____________________________________________________________________________________________________
episode: 94000 loss tensor(0.7786, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5456843169467964
gzsl: unseen=0.1680 , seen=0.5508 , h=0.2574
_

episode: 121000 loss tensor(0.7900, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5499810438518893
gzsl: unseen=0.1728 , seen=0.5808 , h=0.2663
____________________________________________________________________________________________________
episode: 122000 loss tensor(0.7821, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5537722734740301
gzsl: unseen=0.1736 , seen=0.5816 , h=0.2674
____________________________________________________________________________________________________
episode: 123000 loss tensor(0.8063, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5555415139643625
gzsl: unseen=0.1748 , seen=0.5823 , h=0.2689
____________________________________________________________________________________________________
episode: 124000 loss tensor(0.7864, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5526349045873878
gzsl: unseen=0.1755 , seen=0.5799 , h=0.26

...

episode: 361000 loss tensor(0.7693, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5767723998483508
gzsl: unseen=0.1853 , seen=0.6561 , h=0.2889
____________________________________________________________________________________________________
episode: 362000 loss tensor(0.8073, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.577277897131303
gzsl: unseen=0.1845 , seen=0.6564 , h=0.2880
____________________________________________________________________________________________________
episode: 363000 loss tensor(0.7914, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5761405282446607
gzsl: unseen=0.1856 , seen=0.6559 , h=0.2894
____________________________________________________________________________________________________
episode: 364000 loss tensor(0.7660, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5763932768861367
gzsl: unseen=0.1859 , seen=0.6550 , h=0.289

episode: 391000 loss tensor(0.7640, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5766460255276128
gzsl: unseen=0.1845 , seen=0.6605 , h=0.2884
____________________________________________________________________________________________________
episode: 392000 loss tensor(0.7747, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5752559079994946
gzsl: unseen=0.1854 , seen=0.6601 , h=0.2895
____________________________________________________________________________________________________
episode: 393000 loss tensor(0.7520, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5761405282446607
gzsl: unseen=0.1867 , seen=0.6581 , h=0.2908
____________________________________________________________________________________________________
episode: 394000 loss tensor(0.7693, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5751295336787565
gzsl: unseen=0.1869 , seen=0.6584 , h=0.29

episode: 421000 loss tensor(0.7647, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5758877796031846
gzsl: unseen=0.1851 , seen=0.6607 , h=0.2892
____________________________________________________________________________________________________
episode: 422000 loss tensor(0.7787, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5758877796031846
gzsl: unseen=0.1853 , seen=0.6596 , h=0.2893
____________________________________________________________________________________________________
episode: 423000 loss tensor(0.7950, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5755086566409705
gzsl: unseen=0.1856 , seen=0.6598 , h=0.2898
____________________________________________________________________________________________________
episode: 424000 loss tensor(0.7897, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5753822823202325
gzsl: unseen=0.1856 , seen=0.6603 , h=0.28

episode: 451000 loss tensor(0.7816, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5756350309617085
gzsl: unseen=0.1854 , seen=0.6615 , h=0.2896
____________________________________________________________________________________________________
episode: 452000 loss tensor(0.7765, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5756350309617085
gzsl: unseen=0.1858 , seen=0.6612 , h=0.2900
____________________________________________________________________________________________________
episode: 453000 loss tensor(0.7710, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5746240363958044
gzsl: unseen=0.1854 , seen=0.6608 , h=0.2896
____________________________________________________________________________________________________
episode: 454000 loss tensor(0.7734, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5747504107165424
gzsl: unseen=0.1849 , seen=0.6618 , h=0.28

episode: 481000 loss tensor(0.7815, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5753822823202325
gzsl: unseen=0.1865 , seen=0.6630 , h=0.2911
____________________________________________________________________________________________________
episode: 482000 loss tensor(0.7406, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5761405282446607
gzsl: unseen=0.1865 , seen=0.6627 , h=0.2911
____________________________________________________________________________________________________
episode: 483000 loss tensor(0.7717, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5755086566409705
gzsl: unseen=0.1867 , seen=0.6625 , h=0.2913
____________________________________________________________________________________________________
episode: 484000 loss tensor(0.7740, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5756350309617085
gzsl: unseen=0.1868 , seen=0.6625 , h=0.29